<a href="https://colab.research.google.com/github/YonggunJung/AI_Poly/blob/main/02_%EA%B0%9D%EC%B2%B4_%ED%83%90%EC%A7%80(Object_Detection)_YOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 객체 검출(Object Detection)
일반적으로 이미지를 입력 받아 경계상자와 객체 클래스 리스트 출력
예측 클래스와 클래스 신뢰도 출력

## Applications
자율 주행에서 다른 차와 보행자 찾을 때
의료는 종양이나 위험 조직
제조업은 제품
보안에서는 위협 이나 사람

## Bounding Box
이미지에서 하나의 객체 전체를 포함하는 가장 작은 사각형

## IOU(intersection Over Union)
실측값(Ground Truth)과 모델이 예측한 값이 얼마나 겹치는지 나타내는 지표
높을수록 좋음

## NMS(Non-Maximum Suppression, 비최댓값 억제)
확률이 가장 높은 상자와 겹치는 상자들을 제거하는 과장
최댓값을 갖지 않는 상자들을 제거
1. 확률 기준으로 모든 상자를 정렬하고 먼저 가장 높은 확률이 높은 상자를 취함
2. 각 상자에 대해 다른 모든 상자와의 IOU를 계산
3. 특정 임곗값을 넘는 상자는 제거

# 모델 성능평가
### 정률도 재현율

정밀도는 precision 얼마나 잘 검출 하는가

재현율 recall 검출 한거 중에 얼마나 잘 맞는가

## Dataset
VOC는 11만장 20개 클래스


COCO 20만장 80개 카테고리

YOLOv8 Inference 연습
`ultralytics` 라이브러리 설치

In [2]:
%pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 744.8/744.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 67.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 77.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [4]:
# 하드웨어 상태 확인
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.41 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 28.9/78.2 GB disk)


In [5]:
# YOLOv8n은 다양한 작업에 대해 욜로 명령을 사용하여 명령줄 인터페이스에서 직접 사용 할수 있으며 추가 인수를 허용
# Run inference on an image with yolov8n
!yolo predict model = yolov8n.pt source = 'https://ultralytics.com/images/zidane.jpg'

100% 6.23M/6.23M [00:00<00:00, 81.6MB/s]
Ultralytics YOLOv8.1.41 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.20GHz)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

100% 165k/165k [00:00<00:00, 7.62MB/s]
image 1/1 /content/zidane.jpg: 384x640 2 persons, 1 tie, 376.8ms
Speed: 29.5ms preprocess, 376.8ms inference, 32.3ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict
💡 Learn more at https://docs.ultralytics.com/modes/predict


# 2. 파이썬 사용법

In [7]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n.yaml')   # 새 모델을 만든다.
model = YOLO('yolov8n.pt')    #  학습된 모델을 가져온다.

# 모델 사용
results = model.train(data = 'coco128.yaml', epochs = 3)  # 데이터를 추가해서 모델을 학습 시킨다.
results = model.val()   # 검증 데이터로 모델 성능 평가
results = model('https://ultralytics.com/images/bus.jpg')   # 이미지와 비교해본다.
results = model.export(format = 'onnx')     # onnx로 내보내기

Ultralytics YOLOv8.1.41 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=coco128.yaml, epochs=3, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_

train: Scanning /content/datasets/coco128/labels/train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/datasets/coco128/labels/train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]

Plotting labels to runs/detect/train3/labels.jpg... 


...

                   all        128        929      0.682      0.554      0.634      0.474
                person        128        254      0.825      0.657      0.773      0.541
               bicycle        128          6      0.495      0.333      0.327      0.282
                   car        128         46      0.893      0.217      0.281      0.185
            motorcycle        128          5      0.673      0.827      0.938      0.742
              airplane        128          6      0.813      0.833      0.927      0.717
                   bus        128          7       0.74      0.714      0.729      0.657
                 train        128          3       0.56      0.667      0.775      0.687
                 truck        128         12          1      0.367      0.512      0.341
                  boat        128          6      0.408      0.241       0.42      0.283
         traffic light        128         14      0.744      0.209      0.201      0.138
             stop sig

val: Scanning /content/datasets/coco128/labels/train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:35<00:00,  4.38s/it]


                   all        128        929      0.663      0.566      0.631      0.472
                person        128        254      0.812      0.665      0.773      0.544
               bicycle        128          6      0.481      0.333      0.327      0.282
                   car        128         46      0.835      0.217      0.281      0.185
            motorcycle        128          5      0.679       0.85      0.898      0.718
              airplane        128          6      0.796      0.833      0.927      0.717
                   bus        128          7      0.657      0.714      0.728      0.657
                 train        128          3      0.551      0.667      0.775      0.687
                 truck        128         12      0.943      0.417      0.492      0.286
                  boat        128          6      0.306      0.229      0.354      0.187
         traffic light        128         14      0.741      0.207      0.205       0.14
             stop sig

100%|██████████| 476k/476k [00:00<00:00, 11.0MB/s]


image 1/1 /content/bus.jpg: 640x480 4 persons, 1 bus, 1 stop sign, 220.5ms
Speed: 4.6ms preprocess, 220.5ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 480)
Ultralytics YOLOv8.1.41 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.20GHz)

PyTorch: starting from 'runs/detect/train3/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (6.2 MB)
requirements: Ultralytics requirement ['onnx>=1.12.0'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 52.9 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 12.5s, installed 1 package: ['onnx>=1.12.0']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.16.0 opset 17...
ONNX: export success ✅ 13.8s, saved as 'runs/detect/train3/weights/best.onnx' (12.2 MB)

Export complete (15.6s)
Results saved to /content/runs/detect/train3/weights
Predict:         yolo predict task=detect model=run

# TASK

In [8]:
from ultralytics import YOLO

model = YOLO('yolov8n-cls.pt')    # 분류 모델 불러오기
model.train(data='mnist160', epochs = 10)   # 학습된 모델을 가져와서 추가 학습
model('https://ultralytics.com/images/bus.jpg')  # 예측

100%|██████████| 5.30M/5.30M [00:00<00:00, 48.0MB/s]


Ultralytics YOLOv8.1.41 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=mnist160, epochs=10, time=None, patience=100, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, lin

100%|██████████| 70.0k/70.0k [00:00<00:00, 5.93MB/s]
Unzipping /content/datasets/mnist160.zip to /content/datasets/mnist160...: 100%|██████████| 184/184 [00:00<00:00, 3380.46file/s]

Dataset download success ✅ (0.5s), saved to /content/datasets/mnist160

train: /content/datasets/mnist160/train... found 80 images in 10 classes ✅ 
val: None...
test: /content/datasets/mnist160/test... found 80 images in 10 classes ✅ 
Overriding model.yaml nc=1000 with nc=10

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  u

YOLOv8n-cls summary: 99 layers, 1451098 parameters, 1451098 gradients, 3.4 GFLOPs
Transferred 156/158 items from pretrained weights
TensorBoard: Start with 'tensorboard --logdir runs/classify/train', view at http://localhost:6006/


...

                   all      0.338        0.8



10 epochs completed in 0.027 hours.
Optimizer stripped from runs/classify/train/weights/last.pt, 3.0MB
Optimizer stripped from runs/classify/train/weights/best.pt, 3.0MB

Validating runs/classify/train/weights/best.pt...
Ultralytics YOLOv8.1.41 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.20GHz)
YOLOv8n-cls summary (fused): 73 layers, 1447690 parameters, 0 gradients, 3.3 GFLOPs
WARNING ⚠️ Dataset 'split=val' not found, using 'split=test' instead.
train: /content/datasets/mnist160/train... found 80 images in 10 classes ✅ 
val: None...
test: /content/datasets/mnist160/test... found 80 images in 10 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 3/3 [00:02<00:00,  1.38it/s]


                   all      0.338        0.8
Speed: 0.0ms preprocess, 16.9ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train
Results saved to runs/classify/train

Found https://ultralytics.com/images/bus.jpg locally at bus.jpg
image 1/1 /content/bus.jpg: 224x224 8 0.22, 3 0.14, 6 0.13, 0 0.12, 7 0.08, 22.6ms
Speed: 11.1ms preprocess, 22.6ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)


[ultralytics.engine.results.Results object with attributes:
 
 boxes: None
 keypoints: None
 masks: None
 names: {0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9'}
 obb: None
 orig_img: array([[[122, 148, 172],
         [120, 146, 170],
         [125, 153, 177],
         ...,
         [157, 170, 184],
         [158, 171, 185],
         [158, 171, 185]],
 
        [[127, 153, 177],
         [124, 150, 174],
         [127, 155, 179],
         ...,
         [158, 171, 185],
         [159, 172, 186],
         [159, 172, 186]],
 
        [[128, 154, 178],
         [126, 152, 176],
         [126, 154, 178],
         ...,
         [158, 171, 185],
         [158, 171, 185],
         [158, 171, 185]],
 
        ...,
 
        [[185, 185, 191],
         [182, 182, 188],
         [179, 179, 185],
         ...,
         [114, 107, 112],
         [115, 105, 111],
         [116, 106, 112]],
 
        [[157, 157, 163],
         [180, 180, 186],
         [185, 186, 190],


# 2. 검출

In [3]:
from ultralytics import YOLO
model = YOLO('yolov8n.pt')    # 모델 이름 뒤에 아무것도 없으면 보통 디텍션 모델임
model.train(data='coco128.yaml', epochs = 10)   # 학습된 모델을 가져와서 추가 학습
model('https://ultralytics.com/images/bus.jpg')  # 예측

100%|██████████| 6.23M/6.23M [00:00<00:00, 92.6MB/s]


Ultralytics YOLOv8.1.41 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=coco128.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, li

100%|██████████| 6.66M/6.66M [00:00<00:00, 72.4MB/s]
Unzipping /content/datasets/coco128.zip to /content/datasets/coco128...: 100%|██████████| 263/263 [00:00<00:00, 839.84file/s]

Dataset download success ✅ (1.7s), saved to /content/datasets



100%|██████████| 755k/755k [00:00<00:00, 24.2MB/s]


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:07<00:00,  1.99s/it]


                   all        128        929      0.722      0.628      0.711      0.527
                person        128        254      0.864      0.649      0.775      0.565
               bicycle        128          6          1      0.327      0.381      0.317
                   car        128         46      0.733      0.217      0.309      0.192
            motorcycle        128          5      0.777        0.8       0.92      0.724
              airplane        128          6      0.731      0.911      0.955      0.875
                   bus        128          7        0.8      0.714      0.699      0.647
                 train        128          3      0.576          1      0.863      0.807
                 truck        128         12      0.896        0.5      0.544      0.358
                  boat        128          6      0.514      0.333      0.529      0.366
         traffic light        128         14      0.725      0.193      0.201      0.138
             stop sig

100%|██████████| 476k/476k [00:00<00:00, 17.6MB/s]

image 1/1 /content/bus.jpg: 640x480 4 persons, 1 bus, 18.6ms
Speed: 3.3ms preprocess, 18.6ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 480)


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted p

## 3. 분할

In [5]:
from ultralytics import YOLO
model = YOLO('yolov8n-seg.pt')    # 이건 세그멘테이션
model.train(data='coco128-seg.yaml', epochs = 10)   # 학습된 모델을 가져와서 추가 학습
model('https://ultralytics.com/images/bus.jpg')  # 예측

100%|██████████| 6.73M/6.73M [00:00<00:00, 113MB/s]


Ultralytics YOLOv8.1.41 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=coco128-seg.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes

100%|██████████| 6.79M/6.79M [00:00<00:00, 146MB/s]
Unzipping /content/datasets/coco128-seg.zip to /content/datasets/coco128-seg...: 100%|██████████| 263/263 [00:00<00:00, 3375.68file/s]

Dataset download success ✅ (1.3s), saved to /content/datasets


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1 

 12                  -1  1    148224  ultralytics.nn.modules.block.C2f             [384, 128, 1]                 
 13                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 14             [-1, 4]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 15                  -1  1     37248  ultralytics.nn.modules.block.C2f             [192, 64, 1]                  
 16                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
 17            [-1, 12]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 18                  -1  1    123648  ultralytics.nn.modules.block.C2f             [192, 128, 1]                 
 19                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
 20             [-1, 9]  1         0  ultralytics.nn.modules.conv.Concat           [1]  

...

                   all        128        929      0.661      0.583       0.67      0.498      0.748      0.512      0.617      0.403



10 epochs completed in 0.019 hours.
Optimizer stripped from runs/segment/train/weights/last.pt, 7.1MB
Optimizer stripped from runs/segment/train/weights/best.pt, 7.1MB

Validating runs/segment/train/weights/best.pt...
Ultralytics YOLOv8.1.41 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n-seg summary (fused): 195 layers, 3404320 parameters, 0 gradients, 12.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:07<00:00,  1.77s/it]


                   all        128        929      0.659      0.583      0.668      0.496      0.741      0.509      0.601      0.401
                person        128        254       0.84      0.665      0.781      0.549      0.879      0.567       0.75      0.404
               bicycle        128          6      0.563      0.167      0.341       0.26      0.654      0.167      0.345      0.184
                   car        128         46       0.79      0.152      0.344       0.18      0.853      0.126      0.261      0.119
            motorcycle        128          5      0.742        0.8      0.938      0.755      0.795        0.8      0.938      0.479
              airplane        128          6      0.676      0.833      0.882      0.806      0.628      0.667      0.753      0.578
                   bus        128          7      0.871      0.714      0.728      0.576      0.896      0.714       0.72      0.571
                 train        128          3       0.77          1   

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: ultralytics.engine.results.Masks object
 names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 

# 4. 자세추정


In [7]:
from ultralytics import YOLO
model = YOLO('yolov8n-pose.pt')    # 이건 자세
model.train(data='coco8-pose.yaml', epochs = 10)   # 학습된 모델을 가져와서 추가 학습
model('https://ultralytics.com/images/bus.jpg')  # 예측

Ultralytics YOLOv8.1.41 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=pose, mode=train, model=yolov8n-pose.pt, data=coco8-pose.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=T

100%|██████████| 334k/334k [00:00<00:00, 13.9MB/s]
Unzipping /content/datasets/coco8-pose.zip to /content/datasets/coco8-pose...: 100%|██████████| 27/27 [00:00<00:00, 3868.76file/s]

Dataset download success ✅ (1.3s), saved to /content/datasets


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1 

YOLOv8n-pose summary: 250 layers, 3295470 parameters, 3295454 gradients, 9.3 GFLOPs

Transferred 397/397 items from pretrained weights
TensorBoard: Start with 'tensorboard --logdir runs/pose/train2', view at http://localhost:6006/
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /content/datasets/coco8-pose/labels/train... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<00:00, 2484.78it/s]

...

                   all          4         14      0.928      0.918      0.955      0.692      0.977      0.571       0.63       0.33



10 epochs completed in 0.009 hours.
Optimizer stripped from runs/pose/train2/weights/last.pt, 6.8MB
Optimizer stripped from runs/pose/train2/weights/best.pt, 6.8MB

Validating runs/pose/train2/weights/best.pt...
Ultralytics YOLOv8.1.41 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n-pose summary (fused): 187 layers, 3289964 parameters, 0 gradients, 9.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 18.04it/s]


                   all          4         14      0.928      0.921      0.955      0.692      0.977      0.571       0.63      0.329
Speed: 0.3ms preprocess, 4.7ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/pose/train2

Found https://ultralytics.com/images/bus.jpg locally at bus.jpg
image 1/1 /content/bus.jpg: 640x480 4 persons, 31.2ms
Speed: 2.2ms preprocess, 31.2ms inference, 2.6ms postprocess per image at shape (1, 3, 640, 480)


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: ultralytics.engine.results.Keypoints object
 masks: None
 names: {0: 'person'}
 obb: None
 orig_img: array([[[122, 148, 172],
         [120, 146, 170],
         [125, 153, 177],
         ...,
         [157, 170, 184],
         [158, 171, 185],
         [158, 171, 185]],
 
        [[127, 153, 177],
         [124, 150, 174],
         [127, 155, 179],
         ...,
         [158, 171, 185],
         [159, 172, 186],
         [159, 172, 186]],
 
        [[128, 154, 178],
         [126, 152, 176],
         [126, 154, 178],
         ...,
         [158, 171, 185],
         [158, 171, 185],
         [158, 171, 185]],
 
        ...,
 
        [[185, 185, 191],
         [182, 182, 188],
         [179, 179, 185],
         ...,
         [114, 107, 112],
         [115, 105, 111],
         [116, 106, 112]],
 
        [[157, 157, 163],
         [180, 180, 186],
         [185, 186,

# 5. 지향성 경계상자 (OBB)

In [9]:
from ultralytics import YOLO
model = YOLO('yolov8n-obb.pt')    # 이건 자세
model.train(data='coco8-dota.yaml', epochs = 10)   # 학습된 모델을 가져와서 추가 학습
model('https://ultralytics.com/images/bus.jpg')  # 예측

Ultralytics YOLOv8.1.41 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=obb, mode=train, model=yolov8n-obb.pt, data=coco8-dota.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=1024, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=Tr

RuntimeError: Dataset 'coco8-dota.yaml' error ❌ 'coco8-dota.yaml' does not exist